Домашнее задание по теме «Градиентный бустинг»

    Задача 1. Классификация

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, roc_auc_score
from sklearn.metrics import (
    accuracy_score,  # Метрика точности для классификации
    mean_squared_error,  # Средняя квадратичная ошибка для регрессии
    r2_score,  # Коэффициент детерминации для регрессии
    classification_report  # Отчет о классификации
)
import xgboost as xgb

Загрузим данные

In [2]:
train_df=pd.read_csv('train_dataset.csv')

In [3]:
pd.options.display.float_format = '{:,.2f}'.format
train_df.head()

,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,relaxation,...,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries,smoking
0,35,170,85,97.00,0.90,0.90,1,1,118,78,...,70,142,19.80,1,1.00,61,115,125,1,1
1,20,175,110,110.00,0.70,0.90,1,1,119,79,...,71,114,15.90,1,1.10,19,25,30,1,0
2,45,155,65,86.00,0.90,0.90,1,1,110,80,...,57,112,13.70,3,0.60,1090,1400,276,0,0
3,45,165,80,94.00,0.80,0.70,1,1,158,88,...,46,91,16.90,1,0.90,32,36,36,0,0
4,20,165,60,81.00,1.50,0.10,1,1,109,64,...,47,92,14.90,1,1.20,26,28,15,0,0


In [5]:
train_df.info(memory_usage=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38984 entries, 0 to 38983
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   age                  38984 non-null  int64  
 1   height(cm)           38984 non-null  int64  
 2   weight(kg)           38984 non-null  int64  
 3   waist(cm)            38984 non-null  float64
 4   eyesight(left)       38984 non-null  float64
 5   eyesight(right)      38984 non-null  float64
 6   hearing(left)        38984 non-null  int64  
 7   hearing(right)       38984 non-null  int64  
 8   systolic             38984 non-null  int64  
 9   relaxation           38984 non-null  int64  
 10  fasting blood sugar  38984 non-null  int64  
 11  Cholesterol          38984 non-null  int64  
 12  triglyceride         38984 non-null  int64  
 13  HDL                  38984 non-null  int64  
 14  LDL                  38984 non-null  int64  
 15  hemoglobin           38984 non-null 

В датасете нет пропусков. 

Уменьшим вес датасета.

In [4]:
int_cols = train_df.select_dtypes('int').columns

float_cols = train_df.select_dtypes('float').columns
for col in int_cols:
    train_df[col] = pd.to_numeric(train_df[col], downcast='integer')
for col in float_cols:
    train_df[col] = pd.to_numeric(train_df[col], downcast='float')

In [5]:
train_df.info(memory_usage=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38984 entries, 0 to 38983
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   age                  38984 non-null  int8   
 1   height(cm)           38984 non-null  int16  
 2   weight(kg)           38984 non-null  int16  
 3   waist(cm)            38984 non-null  float32
 4   eyesight(left)       38984 non-null  float32
 5   eyesight(right)      38984 non-null  float32
 6   hearing(left)        38984 non-null  int8   
 7   hearing(right)       38984 non-null  int8   
 8   systolic             38984 non-null  int16  
 9   relaxation           38984 non-null  int16  
 10  fasting blood sugar  38984 non-null  int16  
 11  Cholesterol          38984 non-null  int16  
 12  triglyceride         38984 non-null  int16  
 13  HDL                  38984 non-null  int16  
 14  LDL                  38984 non-null  int16  
 15  hemoglobin           38984 non-null 

# EDA

In [7]:
#Подсчитываем сколько процентов незаполнено столбце
percent_missing = train_df.isnull().sum() * 100 / len(train_df.index)
missing_value_df = pd.DataFrame({'Незаполнено столбце %': percent_missing})

missing_value_df

,Незаполнено столбце %
age,0.00
height(cm),0.00
weight(kg),0.00
waist(cm),0.00
eyesight(left),0.00
eyesight(right),0.00
hearing(left),0.00
hearing(right),0.00
systolic,0.00
relaxation,0.00


In [8]:
#Находим максимальные и минимальные значения
min_max_df = pd.DataFrame({'Максимальное значение': train_df.max(), 'Минимальное значение': train_df.min()})

min_max_df

,Максимальное значение,Минимальное значение
age,85.00,20.00
height(cm),190.00,130.00
weight(kg),135.00,30.00
waist(cm),129.00,51.00
eyesight(left),9.90,0.10
eyesight(right),9.90,0.10
hearing(left),2.00,1.00
hearing(right),2.00,1.00
systolic,233.00,71.00
relaxation,146.00,40.00


In [9]:
#Находим средние значения
mean_df = pd.DataFrame({'Среднее значение': train_df.mean()})

mean_df

,Среднее значение
age,44.13
height(cm),164.69
weight(kg),65.94
waist(cm),82.06
eyesight(left),1.01
eyesight(right),1.01
hearing(left),1.03
hearing(right),1.03
systolic,121.48
relaxation,75.99


In [10]:
#Находим медиану
median_df = pd.DataFrame({'Медиана': train_df.median()})

median_df

,Медиана
age,40.00
height(cm),165.00
weight(kg),65.00
waist(cm),82.00
eyesight(left),1.00
eyesight(right),1.00
hearing(left),1.00
hearing(right),1.00
systolic,120.00
relaxation,76.00


In [11]:
#Находим дисперсию
var_df = pd.DataFrame({'Диспрерсия': train_df.var()})

var_df

,Диспрерсия
age,145.53
height(cm),84.41
weight(kg),166.32
waist(cm),86.99
eyesight(left),0.25
eyesight(right),0.24
hearing(left),0.02
hearing(right),0.03
systolic,186.15
relaxation,93.29


In [12]:
#Находим квантиль 0.1 и 0.9
quantile_df = pd.DataFrame({'Квантиль 0.1': train_df.quantile(0.1), 'Квантиль 0.9': train_df.quantile(0.9)})

quantile_df

,Квантиль 0.1,Квантиль 0.9
age,30.00,60.00
height(cm),150.00,175.00
weight(kg),50.00,85.00
waist(cm),70.00,94.00
eyesight(left),0.60,1.50
eyesight(right),0.60,1.50
hearing(left),1.00,1.00
hearing(right),1.00,1.00
systolic,104.00,138.00
relaxation,64.00,88.00


In [13]:
#Находим квартиль 1 и 3
quartile_df = pd.DataFrame({'Квартиль 1': train_df.quantile(0.25), 'Квартиль 3': train_df.quantile(0.75)})

quartile_df

,Квартиль 1,Квартиль 3
age,40.00,55.00
height(cm),160.00,170.00
weight(kg),55.00,75.00
waist(cm),76.00,88.00
eyesight(left),0.80,1.20
eyesight(right),0.80,1.20
hearing(left),1.00,1.00
hearing(right),1.00,1.00
systolic,112.00,130.00
relaxation,70.00,82.00


# Обучение алгоритма XGBClassifier

In [19]:
my_random_state = 42

In [20]:
X = train_df.drop('smoking', axis=1)
Y = train_df['smoking']

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.3, random_state = my_random_state)

Функция для оценки качества алгоритма

In [22]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score, classification_report


def evaluate(model, X_train, X_test, y_train, y_test):
    y_test_pred = model.predict(X_test)
    y_train_pred = model.predict(X_train)

    print("РЕЗУЛЬТАТЫ ОБУЧЕНИЯ (ОБУЧАЮЩАЯ ВЫБОРКА): \n===============================")
    clf_report = pd.DataFrame(classification_report(y_train, y_train_pred, output_dict=True))
    print(f"МАТРИЦА ОШИБОК (CONFUSION MATRIX):\n{confusion_matrix(y_train, y_train_pred)}")
    print(f"ACCURACY ПАРАМЕТР:\n{accuracy_score(y_train, y_train_pred):.4f}")
    print(f"PRECISION ПАРАМЕТР:\n{precision_score(y_train, y_train_pred):.4f}")
    print(f"RECALL ПАРАМЕТР:\n{recall_score(y_train, y_train_pred):.4f}")
    print(f"F1 МЕРА:\n{f1_score(y_train, y_train_pred):.4f}")
    print(f"ОТЧЕТ О КЛАССИФИКАЦИИ:\n{clf_report}")

    print("РЕЗУЛЬТАТЫ ТЕСТОВОЙ ВЫБОРКИ: \n===============================")
    clf_report = pd.DataFrame(classification_report(y_test, y_test_pred, output_dict=True))
    print(f"МАТРИЦА ОШИБОК (CONFUSION MATRIX):\n{confusion_matrix(y_test, y_test_pred)}")
    print(f"ACCURACY ПАРАМЕТР:\n{accuracy_score(y_test, y_test_pred):.4f}")
    print(f"PRECISION ПАРАМЕТР:\n{precision_score(y_test, y_test_pred):.4f}")
    print(f"RECALL ПАРАМЕТР:\n{recall_score(y_test, y_test_pred):.4f}")
    print(f"F1 МЕРА:\n{f1_score(y_test, y_test_pred):.4f}")
    print(f"ОТЧЕТ О КЛАССИФИКАЦИИ:\n{clf_report}")

Обучим алгоритм XGBClassifier

In [46]:
import xgboost as xgb

xgboost_clf = xgb.XGBClassifier()
xgboost_clf.fit(X_train, y_train) #тренировка на данных
evaluate(xgboost_clf, X_train, X_test, y_train, y_test)

РЕЗУЛЬТАТЫ ОБУЧЕНИЯ (ОБУЧАЮЩАЯ ВЫБОРКА): 
МАТРИЦА ОШИБОК (CONFUSION MATRIX):
[[7151  260]
 [ 297 3987]]
ACCURACY ПАРАМЕТР:
0.9524
PRECISION ПАРАМЕТР:
0.9388
RECALL ПАРАМЕТР:
0.9307
F1 МЕРА:
0.9347
ОТЧЕТ О КЛАССИФИКАЦИИ:
                 0        1  accuracy  macro avg  weighted avg
precision     0.96     0.94      0.95       0.95          0.95
recall        0.96     0.93      0.95       0.95          0.95
f1-score      0.96     0.93      0.95       0.95          0.95
support   7,411.00 4,284.00      0.95  11,695.00     11,695.00
РЕЗУЛЬТАТЫ ТЕСТОВОЙ ВЫБОРКИ: 
МАТРИЦА ОШИБОК (CONFUSION MATRIX):
[[13885  3370]
 [ 3372  6662]]
ACCURACY ПАРАМЕТР:
0.7529
PRECISION ПАРАМЕТР:
0.6641
RECALL ПАРАМЕТР:
0.6639
F1 МЕРА:
0.6640
ОТЧЕТ О КЛАССИФИКАЦИИ:
                  0         1  accuracy  macro avg  weighted avg
precision      0.80      0.66      0.75       0.73          0.75
recall         0.80      0.66      0.75       0.73          0.75
f1-score       0.80      0.66      0.75       0.73        

Без подбора параметров алгоритм предсказывает целевую переменную в 75%.

Подберем гиперпараметры и посмотрим как изменится этот показатель.

Чтобы выбрать гиперпараметры алгоритма воспользуемся GridSearchCV. Узнаем лучшие параметры и тренируем с ними алгоритм.

In [41]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

def findXGBClassifier(X_train, y_train):
    param_grid = {
        'eta': [0.01, 0.05, 0.1, 0.15, 0.2],
        'max_depth': [3, 5, 7, 9],
        'subsample': [0.5, 0.7, 0.9, 1],
        'colsample_bytree': [0.5, 0.7, 0.9, 1],
        'n_estimators': [n for n in np.arange(100, 300, 50)],
    }

    xgb_model = xgb.XGBClassifier(random_state=my_random_state, objective='binary:logistic')

    grid_search = GridSearchCV(estimator = xgb_model, param_grid=param_grid, cv=5, n_jobs=-1, scoring='accuracy')
    grid_search.fit(X_train, y_train)

    # Лучшие гиперпараметры
    best_params = grid_search.best_params_
    best_score = grid_search.best_score_
    print(f"Лучшие гиперпараметры: {best_params}")
    print(f"Лучшая точность на валидационной выборке: {best_score}")
    return grid_search

In [43]:
grid_search = findXGBClassifier(X_train, y_train)

Лучшие гиперпараметры: {'colsample_bytree': 0.7, 'eta': 0.05, 'max_depth': 9, 'n_estimators': np.int64(200), 'subsample': 0.7}
Лучшая точность на валидационной выборке: 0.7586147926464302


In [44]:
# Создаем модель с лучшим параметры
XGB_clf = grid_search.best_estimator_

Оцениваем качество алгоритма.

In [45]:
evaluate(XGB_clf, X_train, X_test, y_train, y_test)

РЕЗУЛЬТАТЫ ОБУЧЕНИЯ (ОБУЧАЮЩАЯ ВЫБОРКА): 
МАТРИЦА ОШИБОК (CONFUSION MATRIX):
[[7275  136]
 [ 145 4139]]
ACCURACY ПАРАМЕТР:
0.9760
PRECISION ПАРАМЕТР:
0.9682
RECALL ПАРАМЕТР:
0.9662
F1 МЕРА:
0.9672
ОТЧЕТ О КЛАССИФИКАЦИИ:
                 0        1  accuracy  macro avg  weighted avg
precision     0.98     0.97      0.98       0.97          0.98
recall        0.98     0.97      0.98       0.97          0.98
f1-score      0.98     0.97      0.98       0.97          0.98
support   7,411.00 4,284.00      0.98  11,695.00     11,695.00
РЕЗУЛЬТАТЫ ТЕСТОВОЙ ВЫБОРКИ: 
МАТРИЦА ОШИБОК (CONFUSION MATRIX):
[[13942  3313]
 [ 3218  6816]]
ACCURACY ПАРАМЕТР:
0.7607
PRECISION ПАРАМЕТР:
0.6729
RECALL ПАРАМЕТР:
0.6793
F1 МЕРА:
0.6761
ОТЧЕТ О КЛАССИФИКАЦИИ:
                  0         1  accuracy  macro avg  weighted avg
precision      0.81      0.67      0.76       0.74          0.76
recall         0.81      0.68      0.76       0.74          0.76
f1-score       0.81      0.68      0.76       0.74        

Модель предсказывает целевую переменную в 76%. Она показывает приемлимую точность. 